# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into deep neural networks for NLP!

## Objectives

You will be able to:

- Train a Word2Vec model and transform words into vectors 
- Obtain most similar words by using methods associated with word vectors 


## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec model. Then, you'll move onto building neural networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [3]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data is stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a GitHub repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json()` function from Pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've imported the data, inspect the first few rows of the DataFrame to see what your data looks like. 

In [5]:
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
df.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data -- `headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `'combined_text'` that consists of the data from the `'headline'` column plus a space character (`' '`) plus the data from the `'short_description'` column 
* Use the `'combined_text'` column's `.map()` method and pass in `word_tokenize`. Store the result returned in `data` 

In [7]:
df['combined_text'] = df['headline'] + ' ' + df['short_description']
data = df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [8]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from `gensim`. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1` 
    * The number of threads to use during training, `workers=4`

In [9]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call the `.train()` method on your model and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count` 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [10]:
model.corpus_count

200853

In [11]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55565145, 67352790)

Great! You now have a fully trained model! The word vectors themselves are stored in the `Word2VecKeyedVectors` instance, which is stored in the `.wv` attribute. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [12]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this by passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [13]:
wv.most_similar('Texas')

[('Oregon', 0.8398951292037964),
 ('Maryland', 0.8085190057754517),
 ('Georgia', 0.8055722713470459),
 ('Connecticut', 0.8008735179901123),
 ('Ohio', 0.8004444241523743),
 ('Pennsylvania', 0.7905731201171875),
 ('Illinois', 0.7881308794021606),
 ('Oklahoma', 0.7876158356666565),
 ('Massachusetts', 0.780232310295105),
 ('Louisiana', 0.7768672704696655)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `.most_similar()` method's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [14]:
wv.most_similar(negative='Texas')

[('Ex-Lions', 0.42663487792015076),
 ('Double-parked', 0.416659951210022),
 ('Uglies', 0.3958323001861572),
 ('Animal-rescue', 0.38633137941360474),
 ('Shukri', 0.38230979442596436),
 ('Hendrix-Jackson', 0.37844789028167725),
 ('Disney-esque', 0.3759292960166931),
 ('Parent/Grandparent', 0.3715026080608368),
 ('petrifying', 0.37066906690597534),
 ('Slammers', 0.3618544936180115)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [15]:
wv['Texas']

array([ 0.08451179, -1.0921831 , -2.1178317 , -0.9946147 , -1.3912519 ,
        0.6557976 ,  1.597171  ,  0.4409616 ,  1.4041866 , -3.5051322 ,
       -0.83137834,  0.30996385, -1.2257903 ,  0.8892361 , -0.1820558 ,
        0.23073366, -0.24855782, -0.18924187, -1.9884943 ,  1.6588657 ,
        0.8483266 , -0.26941848,  3.362963  , -0.42438364,  1.3129929 ,
        1.9573019 , -3.2230926 , -0.0493625 , -0.27553716, -0.75745165,
       -1.0500238 , -1.5435413 ,  0.91489047, -0.6748221 , -2.5529912 ,
        0.1834732 ,  1.1349826 ,  0.9432927 ,  0.16650158,  1.2307965 ,
        1.4827318 , -0.24457479,  1.0394863 ,  0.22369595,  1.1496941 ,
       -0.11767153,  0.6512034 ,  0.95902336,  1.2708286 , -2.1373496 ,
        0.24546508, -0.2046515 ,  1.4112761 ,  0.5723272 , -1.5529535 ,
        2.2295792 , -0.14068705, -2.4861314 , -1.8818414 , -1.2921313 ,
        1.1098229 ,  0.56001353,  0.38919464,  1.475564  , -0.21391714,
       -0.04454534, -0.9167466 , -0.9392887 ,  1.660436  ,  1.13

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [16]:
wv.vectors

array([[-1.0251313e+00, -3.3530886e+00, -1.8965150e-01, ...,
        -6.6283512e-01,  9.7946334e-01, -9.6975133e-02],
       [-2.2788363e+00, -2.8416622e+00,  1.7030842e+00, ...,
        -3.0935521e+00, -5.9981275e-01, -1.5951482e+00],
       [-5.8863258e-01, -1.2344606e+00, -7.3102891e-01, ...,
        -2.9668539e+00, -2.4804048e-01, -9.0662950e-01],
       ...,
       [ 1.2122246e-01, -1.8310819e-02,  4.5250628e-02, ...,
         2.1638109e-01, -4.4229127e-02,  7.4393149e-03],
       [ 1.9924039e-02, -4.5767646e-02,  6.3645899e-02, ...,
         5.1135901e-02, -3.4511213e-03,  1.7083393e-02],
       [ 6.2887110e-02, -4.4760633e-02,  2.3371700e-02, ...,
         8.9054987e-02, -1.0461380e-02,  6.0219382e-04]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `.most_similar()` method and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [17]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('princess', 0.6326525211334229),
 ('crown', 0.5764867067337036),
 ('queen', 0.5678408145904541),
 ('monster', 0.5647764205932617),
 ('ring', 0.5548943877220154),
 ('revival', 0.5527797937393188),
 ('lover', 0.546748697757721),
 ('brunette', 0.5462658405303955),
 ('jewel', 0.536015510559082),
 ('fan', 0.5341992974281311)]

As you can see from the output above, your model isn't perfect, but 'Queen' and 'Princess' are still in the top 5. As you can see from the other word in top 5, 'reminiscent' -- your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to create vectorized word embeddings!